In [12]:

from misc.visualizer import *

import numpy as np
import math
import time
import logging
from data.data_loader import Dataset
from misc.preferences import PREFERENCES
from misc.run_configuration import get_default_params, OutputLayerType, LearningSchedulerType, OptimizerType, hyperOpt_goodParams, default_params
from misc import utils

from optimizer import get_optimizer
from criterion import NllLoss, LossCombiner

from models.transformer.encoder import TransformerEncoder
from models.jointAspectTagger import JointAspectTagger
from trainer.train import Trainer
import pprint
from data.germeval2017 import germeval2017_dataset as dsl

import torch
import pprint

import torchtext
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [2]:
def load_model(dataset, rc, experiment_name):
    loss = LossCombiner(4, dataset.class_weights, NllLoss)
    transformer = TransformerEncoder(dataset.source_embedding,
                                     hyperparameters=rc)
    model = JointAspectTagger(transformer, rc, 4, 20, dataset.target_names)
    optimizer = get_optimizer(model, rc)
    trainer = Trainer(
                        model,
                        loss,
                        optimizer,
                        rc,
                        dataset,
                        experiment_name,
                        enable_tensorboard=False,
                        verbose=False)
    return trainer

def load_dataset(rc, logger, task):
    dataset = Dataset(
        task,
        logger,
        rc,
        source_index=PREFERENCES.source_index,
        target_vocab_index=PREFERENCES.target_vocab_index,
        data_path=PREFERENCES.data_root,
        train_file=PREFERENCES.data_train,
        valid_file=PREFERENCES.data_validation,
        test_file=PREFERENCES.data_test,
        file_format=PREFERENCES.file_format,
        init_token=None,
        eos_token=None
    )
    dataset.load_data(dsl, verbose=False)
    return dataset

In [3]:
PREFERENCES.defaults(
    data_root='./data/data/germeval2017',
    data_train='train_v1.4.tsv',    
    data_validation='dev_v1.4.tsv',
    data_test='test_TIMESTAMP1.tsv',
    source_index=0,
    target_vocab_index=2,
    file_format='csv'
)
main_experiment_name = 'GermEval7_Experiments'
use_cuda = True

In [4]:
baseline = {**default_params, **hyperOpt_goodParams}
print(pprint.pformat(baseline, indent=2))
utils.get_current_git_commit()
print('Current commit: ' + utils.get_current_git_commit())

{ 'att_d_k': 300,
  'att_d_v': 300,
  'batch_size': 12,
  'clip_comments_to': 113,
  'dropout_rate': 0.302424,
  'early_stopping': 5,
  'embedding_dim': 300,
  'embedding_name': '6B',
  'embedding_type': 'fasttext',
  'harmonize_bahn': True,
  'language': 'de',
  'learning_rate_scheduler': { 'noam_learning_rate_factor': 1.418,
                               'noam_learning_rate_warmup': 8000},
  'learning_rate_scheduler_type': <LearningSchedulerType.Noam: 1>,
  'log_every_xth_iteration': -1,
  'model_size': 300,
  'num_encoder_blocks': 2,
  'num_epochs': 25,
  'num_heads': 1,
  'optimizer': { 'adam_beta1': 0.81,
                 'adam_beta2': 0.7173,
                 'adam_eps': 0.000814,
                 'learning_rate': 7.2e-05},
  'optimizer_type': <OptimizerType.Adam: 1>,
  'organic_text_cleaning': False,
  'output_dropout_rate': 0.79602089766246,
  'output_layer': { 'output_conv_kernel_size': 5,
                    'output_conv_num_filters': 300,
                    'output_conv_pa

In [9]:
experiment_name = utils.create_loggers(experiment_name=main_experiment_name)
logger = logging.getLogger(__name__)
dataset_logger = logging.getLogger('data_loader')
rc = get_default_params(use_cuda=True, overwrite={}, from_default=baseline)

dataset = load_dataset(rc, dataset_logger, rc.task)
logger.debug('dataset loaded')


Log path is  /Users/felix/Documents/Repositories/TUM/ABSA-Transformer/logs/GermEval7_Experiments/20190401/5
unk UNK
unk <pad>
unk 
unk gutefragenettag
unk 
unk 2
unk 1
unk 2015
unk 3
unk 2016
unk 5
unk 10
unk 4
unk dbbahn
unk 20
unk 30
unk 1
unk 21
unk 6
unk 15
unk «
unk 11101977
unk 8
unk 11101977
unk 25
unk 7
unk nasakennedysonnenkönigrivera
unk 12
unk 19
unk ”
unk 2
unk 50
unk 21
unk 100
unk 11
unk 40
unk 9
unk 60
unk 16
unk 2
unk 1
unk 14
unk 18
unk 31
unk 3
unk 500
unk 5
unk 3
unk 0916451107
unk dbinfo
unk 13
unk dpaafx
unk icestrecke
unk 22
unk 850
unk 250
unk 24
unk 10
unk 17
unk 200
unk 08042016
unk 2
unk 2014
unk 23
unk sbahnberlin
unk 29
unk ️
unk 24
unk 4
unk bahnansagen
unk 45
unk 70
unk ’
unk 1000
unk 7
unk 20
unk 8
unk 6
unk 300
unk 4
unk sbahnen
unk «
unk 35
unk 5
unk 4
unk 0
unk hannoverkassel
unk 21052015
unk 2000
unk 31012016
unk sbahnhof
unk 26
unk störticker
unk 28
unk 3
unk 80
unk 30
unk 1
unk 2005
unk onlinewettbewerb
unk 31122015
unk 90
unk astefanowitsch
unk gri

unk publireportage
unk qualifi
unk quartalsergebnis
unk querdurchslandticket
unk railwayculture
unk railwaysde
unk railwaysofourworld
unk rekordstreik
unk roundupaktien
unk rumpenhorst
unk sabrinabianca
unk sachsenfrankenmagistrale
unk sbahnbetrieb
unk sbahnchaosde
unk sbahngesamtnetz
unk sbahnhöfen
unk sbahnstationen
unk sbahntakt
unk sbahntunnels
unk schleuserautos
unk schmidtarchiv
unk smartphoneapp
unk solothurnmoutierbahn
unk sparpreisaktion
unk stadtneurotikr
unk stadtumlandbahn
unk strausbergnord
unk streikpause
unk stresscoach
unk suckt
unk svzde
unk swrde
unk teilausfälle
unk thannmatzbach
unk thescorvo
unk throwbackthursday
unk ticketkontrolle
unk toysstory
unk trickyscarfy
unk türblockierer
unk umweltwettbewerb
unk verkehrsausschussvorsitzender
unk viome
unk vrride
unk vwkäfer
unk wagenreihenfolge
unk webde
unk wendlingenulm
unk widmerschlumpf
unk wozzap
unk xenius
unk ziwhg
unk zuglabor
unk zugteam
unk özf
unk ’
unk ▽
unk ❚
unk ������
unk 112
unk 13
unk 1435
unk 19682
unk 1

unk 362015
unk 3712
unk 380
unk 396
unk 402
unk 412503342135100
unk 42
unk 4230
unk 424409974264289
unk 441
unk 46
unk 4730
unk 4784
unk 485
unk 490
unk 5060
unk 508
unk 5101
unk 515
unk 516
unk 520
unk 521
unk 54
unk 542
unk 5500
unk 55000
unk 556
unk 5671
unk 570114
unk 584863828190950
unk 59
unk 608
unk 61
unk 616
unk 621
unk 623
unk 6270
unk 63
unk 64
unk 643
unk 649
unk 65000
unk 659
unk 66
unk 661
unk 68000
unk 688
unk 690
unk 700000
unk 709
unk 725
unk 745
unk 760
unk 77
unk 799
unk 802
unk 80211
unk 8200
unk 841
unk 842016
unk 845
unk 863
unk 880
unk 887
unk 900
unk 9000
unk 98
unk abfahrtspünktlichkeit
unk abmonatskarte
unk adsbygoogle
unk aleksandrah
unk allinclusiveangebot
unk allnetflat
unk alsisi
unk altbaugüterbahnhof
unk altbaumitlift
unk altenburggera
unk alternativzug
unk anbieterobjektid
unk andenbahn
unk andreasstavri
unk androidoma
unk anob
unk antimobbingprogramm
unk appdb
unk arnhemnijmegen
unk arnostoffels
unk artede
unk arzthelferinmfakrankenschwestertresenkraft

unk 1211
unk 12169
unk 1217
unk 1220
unk 122014
unk 122015
unk 12209
unk 1222
unk 1224
unk 12249
unk 1225
unk 1227302
unk 1234
unk 12355
unk 1237
unk 1241426
unk 12435
unk 1244
unk 12527
unk 12557
unk 1257
unk 12629
unk 1280
unk 1301
unk 1308
unk 13086
unk 13121
unk 131215
unk 13122015
unk 1314
unk 13140
unk 1316
unk 1322
unk 1323
unk 1325
unk 1326
unk 1327
unk 1331
unk 1332
unk 1338
unk 13407
unk 1341
unk 134140
unk 1344
unk 1348
unk 13503
unk 13507
unk 1353
unk 1355
unk 13585
unk 1359
unk 138
unk 14000
unk 1403
unk 14052
unk 141115
unk 14112015
unk 141214
unk 14195
unk 1426
unk 1442
unk 1443
unk 1446
unk 1450
unk 1499
unk 1503
unk 15042016
unk 15052016
unk 1507
unk 1509
unk 1511
unk 151215
unk 15141
unk 15160
unk 1517
unk 1518
unk 152
unk 1527
unk 15344
unk 1540
unk 1542
unk 1544
unk 1545
unk 1549
unk 156
unk 1560
unk 1578188835737980
unk 1600
unk 16122015
unk 1617
unk 1619
unk 1622
unk 1626
unk 1634
unk 1637
unk 1643
unk 16471
unk 1654
unk 1656
unk 1657
unk 1658
unk 1660
unk 1680
un

unk flexiblität
unk flextickets
unk flotts
unk flpg
unk flugbegleitergewerkschaft
unk flugh
unk flughafenmesse
unk flughafensprecher
unk flugradar
unk flugtipps
unk flugtrackern
unk flugverfolgung
unk flugzeugabgase
unk flã
unk flüchtlingshass
unk focusonlineeilmeldungen
unk focusonlinegesundheit
unk focustitel
unk folgetakt
unk folgeverzögerungen
unk folkstammtisch
unk followerpower
unk fooddiary
unk forchheimnord
unk fordpass
unk forensberger
unk foreverlivingproducts
unk fotografierenund
unk fototobi
unk frankenthüringenexpress
unk frankfur
unk frankfurtammain
unk frankfurtbad
unk frankfurthannover
unk frankfurthöchst
unk frankfurtmainhbf
unk frankfurtmein
unk frankfurto
unk frankvde
unk fraumaja
unk fraunachti
unk freiepressede
unk freifahrtticket
unk freitickets
unk freizeitmobilität
unk freudenvollere
unk freyunggrafenau
unk friedrichebertstraße
unk friedrichshafenhafen
unk friedshafenstadt
unk frlokal
unk frmtv
unk frondenberg
unk fronlinede
unk fruchtportal
unk frundsbergstr
un

...

unk oxenkuehn
unk ozeanausliebe
unk ozeanbewohner
unk ozeanblaubeige
unk ozonlinede
unk oöschnitt
unk packetapps
unk packn
unk packpnt
unk packvolumen
unk paddysinternet
unk pailetten
unk paintballkügelchen
unk paketkooperation
unk paketreisen
unk pakos
unk palmrainzubringer
unk pamperroutine
unk pampersfreie
unk panelgag
unk pankowheinersdorf
unk pankownähe
unk pankratowitz
unk pankreaselastase
unk pankreasschübe
unk pannenfahrer
unk pannenkraftwerk
unk pannensichere
unk panoramalounge
unk pantherschwarz
unk pantoffeltier
unk papaalleinzuhaus
unk papierhandtüchern
unk papierkabel
unk pappbecheraber
unk papschachteln
unk paradiside
unk paradoxondie
unk paraebeispiel
unk parakanuweltmeisterschaften
unk parallelfred
unk parfeng
unk paricafé
unk parichm
unk parismit
unk parismünchen
unk parkausweisen
unk parkempfehlung
unk parkgerechtigkeit
unk parkhausentscheid
unk parkplatze
unk parkplatzsystem
unk parkraumbewirtschaftungszone
unk parkraummanagements
unk parkregelung
unk parkride
unk pa

unk rohrpostbahn
unk rolfaxel
unk rolfjoachim
unk rollenspielflut
unk rollerskaterteam
unk rollstuhlsitzend
unk rollstuhlspasti
unk rolltreppenmonteur
unk rolltreppenstehenbleiber
unk rolę
unk romantikwochenende
unk romeocappu
unk romijoxxlp
unk romiroemsn
unk romã
unk rorschachheidenbergbahn
unk rosefahrer
unk rosenkranzschenklötzschpb
unk rosenmarmelade
unk rossmannkundenkarte
unk rostocklaage
unk roteschleifenzug
unk rotesnichts
unk rotrot
unk rotstiftpläne
unk rotweisse
unk rotweisserfurt
unk rotzbesoffen
unk rotzendenschniefenden
unk rotzgören
unk routecalculator
unk routeneingabe
unk rowiro
unk royalnightmare
unk royalwellness
unk royalwellnessshopde
unk roßburg
unk rpckflug
unk rphd
unk rpofalla
unk rpowirtschaft
unk rpten
unk rqanp
unk rquotehardriver
unk rratedversion
unk rrradio
unk rrrockstah
unk rrxprojekt
unk rrxzüge
unk rssfeeds
unk rtartikel
unk rtgthyngen
unk rtig
unk rtroemer
unk rttanyanesia
unk rubeland
unk rubinrotxx
unk rubriklistenbild
unk ruckfreier
unk rucksackd

unk spitzenmanagement
unk spitzentreffens
unk spitzmischling
unk splpltzknd
unk spntan
unk spoileralarm
unk spongib
unk sponsorenlösung
unk sponsoringdebatte
unk sportfrisur
unk sporthighlights
unk sportkegelbundesliga
unk sportkegelnewssportkegelnews
unk sportklamotten
unk sportlerbahn
unk sportlervisitenkarte
unk sportlichergepflegternicht
unk sportlk
unk sportschwimmern
unk sportthies
unk sportvereinsmitglieder
unk sportwahn
unk sportzen
unk spottedde
unk spottersorgen
unk spotterticket
unk sprachehab
unk sprachfaschistinnen
unk sprachund
unk sprachurlaub
unk sprayerutensilien
unk spreadtrum
unk sprechendas
unk spreetaufe
unk spreewaldtunnel
unk sprengstoffdelikten
unk sprengstoffkörper
unk springmausensembles
unk springnichts
unk sprinterzuschlag
unk sprintest
unk sprintgang
unk sprintqualli
unk sprinttriumph
unk spritkrimi
unk sprüchemehr
unk sprüchleins
unk sprühattacke
unk sprüherkollege
unk sprühköpfe
unk spschweiz
unk spuckende
unk spurenwir
unk spurgblog
unk spätman
unk spätu

unk umguckt
unk umkaempfte
unk umkippe
unk umkippst
unk umkleidentür
unk umkr
unk umleitungsbeschilderung
unk umleitungsweg
unk umliegwnden
unk umrennen
unk umrüstpakett
unk umsatzbeteiligt
unk umschlagswerk
unk umsetzungspartner
unk umsonstbahn
unk umsteigendamit
unk umsteigender
unk umsteigenlieber
unk umsteigenund
unk umsteigerbahnhöfen
unk umstiegsbibliotheken
unk umstiegsverbindungen
unk umverteilungsübung
unk umwe
unk umweltfeidliche
unk umweltkarteabc
unk umweltkartegültigkeit
unk umweltkartenbesitzer
unk umweltkrankheiten
unk umweltmenschen
unk umweltnur
unk umweltschweinerei
unk umweltskarte
unk umweltsäue
unk umzerziehung
unk umzuschlagenwas
unk umölen
unk unaktraktiv
unk unaneh
unk unanehmlichkeiten
unk unangenem
unk unarbeitsorganisation
unk unausgebildetsten
unk unbeampelte
unk unbegrapschte
unk unbeka
unk unbekanntdie
unk unbekanntwegen
unk unbeliebtere
unk unbewilligt
unk unboxingbericht
unk uncooles
unk uncre
unk undanderen
unk undasphaltflickstellen
unk undbahnteam
unk

unk wursthanswurst
unk wurstkommission
unk wurstkreb
unk wurststulle
unk wurzelgeschlagenen
unk wuscheln
unk wusstman
unk wutattacke
unk wutbaron
unk wutbriefe
unk wutenbergs
unk wutmütter
unk wuuhu
unk wuunderschön
unk wuää
unk wweparis
unk wwwambergtechnologiesch
unk wwwanleihencheckde
unk wwwbahnde
unk wwwbahndepviewhomekontakterstattungsanfragekulanzshtml
unk wwwbastiarltde
unk wwwdrehscheibeonlinede
unk wwwfacebookcomdeutschebahn
unk wwwfbcomdbkarriere
unk wyvernlord
unk wzde
unk wzonlinede
unk wächterbach
unk wämstlergäßchen
unk wäredas
unk wäredie
unk wärenbahn
unk wäresüleder
unk wäscheabwurf
unk wüansch
unk wülfrathaprath
unk wün
unk wünderschönehellige
unk wünschapos
unk würdenleute
unk würdeohne
unk würdeund
unk würmerfresser
unk würmtalnarhalla
unk würzbrug
unk würzburgfrauenland
unk würzburghbf
unk würzburgnürnbergmit
unk würzburgstadt
unk würzburgsüd
unk würzburgvideovalis
unk würzburgzell
unk wüstentemperaturen
unk xabendessen
unk xalyen
unk xaposd
unk xayuex
unk xboxde


In [ ]:
logger.debug('Load model')
trainer.load_model(custom_path='C:\\Users\\felix\\OneDrive\\Studium\\Studium\\6. Semester\\MA\\Project\\ABSA-Transformer\\logs\\GermEval7_Experiments\\20190401\\0\\checkpoints')
trainer.set_cuda(True)
result = trainer.perform_final_evaluation(use_test_set=True, verbose=False)
print(result)

In [10]:
def produce_test_gold_labels(iterator: torchtext.data.Iterator, dataset: Dataset, filename='evaluation_output.xml'):

    fields = dataset.fields
    with torch.no_grad():
        iterator.init_epoch()
        
        tree = ET.ElementTree()
        root = ET.Element('Documents')

        for batch in iterator:
            doc_id, comment, relevance, aspect_sentiment, general_sentiment = batch.id, batch.comments, batch.relevance, batch.aspect_sentiments, batch.general_sentiments
            doc_id = fields['id'].reverse(doc_id.unsqueeze(1))
            comment = fields['comments'].reverse(comment)
            relevance = ['false' if r == 0 else 'true' for r in relevance]
            general_sentiment = fields['general_sentiments'].reverse(general_sentiment.unsqueeze(1))
            aspect_sentiment = fields['aspect_sentiments'].reverse(aspect_sentiment, detokenize=False)

            for i in range(len(doc_id)):
                docuement_elem = ET.SubElement(root, 'Document', {'id': doc_id[i]})

                rel_field = ET.SubElement(docuement_elem, 'relevance')
                rel_field.text = relevance[i]

                sen_field = ET.SubElement(docuement_elem, 'sentiment')
                sen_field.text = general_sentiment[i]

                text_field = ET.SubElement(docuement_elem, 'text')
                text_field.text = comment[i]

                # don't add aspects if field not relevant
                if relevance[i] == 'false':
                    continue
                options_elem = ET.SubElement(docuement_elem, 'Opinions')

                # add aspects
                for sentiment, a_name in zip(aspect_sentiment[i], dataset.target_names):
                    if sentiment == 'n/a':
                        continue

                    asp_field = ET.SubElement(options_elem, 'Opinion', {
                        'category': a_name,
                        'polarity': sentiment
                    })

        #print(BeautifulSoup(ET.tostring(tree), "xml").prettify())
        tree._setroot(root)
        tree.write(filename, encoding='utf-8')


In [13]:
produce_test_gold_labels(dataset.test_iter, dataset)